In [2]:
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
import re
import json
import numpy as np
import os, sys
import time 

In [3]:
main_dir = os.path.normpath(os.getcwd() + os.sep + os.pardir)

'/Users/evanzappa/SportsBetting'

In [4]:
driver = webdriver.Chrome(main_dir + "/PrizePicks/chromedriver")

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


# Functions

In [127]:
def manipulation(results):
    all_props = []
    prop_lines = []
    for x in results:
        y = x.find('a',href = True)
        final = 'https://www.pinnacle.com' + y['href'] + '#player-props'
        driver.get(final)
        time.sleep(3)
        content = driver.page_source
        soup = BeautifulSoup(content, "html.parser")
        result_1 = soup.find_all("div", class_ = "style_primary__3IwKt style_marketGroup__1-qlF")
        for z in result_1:
            prop_name = z.find("span", class_ = "style_titleText__35WhH")
            prop_line = z.find_all("span", class_ = "style_label__2KJur")
            lines = z.find_all("span", class_ = "style_price__15SlF")
            all_props.append((prop_name,prop_line,lines))
    final = []
    for x in all_props:
        str_x = str(x[0])
        temp_x = str_x.split('>')
        temp_2 = temp_x[1].split('<')
        name_line = temp_2[0].split('(')
        name = name_line[0].strip()
        if len(name_line) <= 1:
            continue
        line = name_line[1]
        line = line.strip(')')
        if len(x[1]) != 2:
            continue
        str_x1 = str(x[1][0])
        temp_x1 = str_x1.split('>')
        temp_2x1 = temp_x1[1].split('<')
        str_x2 = str(x[1][1])
        temp_x2 = str_x2.split('>')
        temp_2x2 = temp_x2[1].split('<')
        over_bet = temp_2x1[0]
        under_bet = temp_2x2[0]
        pin_line = re.findall("\d+\.\d+",over_bet)
        pin_line = float(pin_line[0])



        over_odds = str(x[2][0])
        over_odds = over_odds.split('>')
        over_odds = over_odds[1].split('<')
        over_odds = over_odds[0]

        under_odds = str(x[2][1])
        under_odds = under_odds.split('>')
        under_odds = under_odds[1].split('<')
        under_odds = under_odds[0]  

        final.append({'player_name': name,'prop_type': line, 'pinnacle_line' : pin_line, 
                      'over_price': float(over_odds),'under_price': float(under_odds)})
    final_df = pd.DataFrame(final)
    return final_df

def converting_odds(dataframe):
    conditions_over = [
        dataframe['over_price'] >= 2.00,
        dataframe['over_price'] < 2.00
    ]
    values_over = [
            (dataframe['over_price'] - 1) * 100,
            (-100)/(dataframe['over_price']-1)
    ]
    dataframe['over_price'] = np.select(conditions_over,values_over)
    conditions_under = [
            dataframe['under_price'] >= 2.00,
            dataframe['under_price'] < 2.00
    ]
    values_under = [
            (dataframe['under_price'] - 1) * 100,
            (-100)/(dataframe['under_price']-1)
    ]
    dataframe['under_price'] = np.select(conditions_under,values_under)
    return dataframe

# Scraping NFL

In [128]:
nfl_link = 'https://www.pinnacle.com/en/football/nfl/matchups#period:0'

In [129]:
driver.get(nfl_link)
time.sleep(5)

In [130]:
content = driver.page_source
soup = BeautifulSoup(content, "html.parser")

In [131]:
results = soup.find_all("div", class_="style_metadata__1FIzs")

In [132]:
nfl_final_df = manipulation(results)
nfl_final_df = converting_odds(nfl_final_df)

In [133]:
nfl_final_df['league'] = 'nfl'
nfl_final_df

player_name        prop_type  pinnacle_line  over_price  \
0          Cade Otton    TD Receptions            0.5  360.000000   
1        Chris Godwin   Long Reception           22.5 -113.122172   
2        Chris Godwin    Receiving Yds           71.5 -114.025086   
3        Chris Godwin    TD Receptions            0.5  147.000000   
4      Devin Duvernay    TD Receptions            0.5  395.000000   
5         Gus Edwards          Rush TD            0.5  177.000000   
6         Gus Edwards         Rush Yds           45.5 -120.048019   
7       Justin Tucker   Kicking Points            7.5 -114.025086   
8       Lamar Jackson      Completions           19.5 -106.044539   
9       Lamar Jackson    Interceptions            0.5 -127.064803   
10      Lamar Jackson  Long Completion           33.5 -114.025086   
11      Lamar Jackson    Pass Attempts           29.5 -101.010101   
12      Lamar Jackson         Pass Yds          213.5 -114.025086   
13      Lamar Jackson          Rush TD            0.5  208.000000   
14      Lamar Jackson         Rush Yds           61.5 -116.009281   
15      Lamar Jackson        TD Passes            1.5  143.000000   
16  Leonard Fournette   Long Reception           13.5 -102.040816   
17  Leonard Fournette    Receiving Yds           28.5 -113.122172   
18  Leonard Fournette          Rush TD            0.5  154.000000   
19  Leonard Fournette         Rush Yds           49.5  104.000000   
20  Leonard Fournette    TD Receptions            0.5  335.000000   
21         Mike Evans   Long Reception           24.5 -122.100122   
22         Mike Evans    Receiving Yds           68.5 -113.122172   
23         Mike Evans    TD Receptions            0.5  132.000000   
24      Rachaad White   Long Reception            8.5 -121.065375   
25      Rachaad White    Receiving Yds           12.5  104.000000   
26          Tom Brady    Interceptions            0.5  106.000000   
27          Tom Brady  Long Completion           37.5 -119.047619   
28          Tom Brady         Pass Yds          279.5 -116.009281   
29          Tom Brady        TD Passes            1.5 -124.069479   

    under_price league  
0   -714.285714    nfl  
1   -118.063754    nfl  
2   -116.009281    nfl  
3   -204.081633    nfl  
4   -840.336134    nfl  
5   -256.410256    nfl  
6   -110.011001    nfl  
7   -116.009281    nfl  
8   -125.000000    nfl  
9   -104.058273    nfl  
10  -116.009281    nfl  
11  -131.061599    nfl  
12  -116.009281    nfl  
13  -315.457413    nfl  
14  -114.025086    nfl  
15  -194.174757    nfl  
16  -130.039012    nfl  
17  -117.096019    nfl  
18  -215.053763    nfl  
19  -137.174211    nfl  
20  -636.942675    nfl  
21  -109.051254    nfl  
22  -117.096019    nfl  
23  -179.211470    nfl  
24  -110.011001    nfl  
25  -139.082058    nfl  
26  -142.045455    nfl  
27  -111.111111    nfl  
28  -114.025086    nfl  
29  -108.108108    nfl

# Scraping NBA

In [134]:
nba_link = 'https://www.pinnacle.com/en/basketball/nba/matchups#period:0'

In [135]:
driver.get(nba_link)
time.sleep(5)

In [136]:
content = driver.page_source
soup = BeautifulSoup(content, "html.parser")

In [137]:
results = soup.find_all("div", class_="style_metadata__1FIzs")

In [138]:
nba_final_df = manipulation(results)
nba_final_df = converting_odds(nba_final_df)

In [139]:
nba_final_df['league'] = 'nba'
nba_final_df

player_name      prop_type  pinnacle_line  over_price  \
0                Caris LeVert     3 Point FG            1.5  106.000000   
1                Caris LeVert        Assists            5.5  139.000000   
2                Caris LeVert         Blocks            0.5  262.000000   
3                Caris LeVert         Points           15.5 -115.074799   
4                Caris LeVert  Pts+Rebs+Asts           24.5 -121.065375   
5                Caris LeVert       Rebounds            3.5 -149.031297   
6                Caris LeVert  Steals+Blocks            1.5  153.000000   
7                Caris LeVert      Turnovers            1.5 -159.235669   
8                Cole Anthony     3 Point FG            2.5  178.000000   
9                Cole Anthony        Assists            5.5 -120.048019   
10               Cole Anthony         Blocks            0.5  200.000000   
11               Cole Anthony         Points           16.5  110.000000   
12               Cole Anthony  Pts+Rebs+Asts           26.5 -109.051254   
13               Cole Anthony       Rebounds            4.5  125.000000   
14               Cole Anthony  Steals+Blocks            0.5 -268.096515   
15               Cole Anthony      Turnovers            2.5 -111.111111   
16           Donovan Mitchell     3 Point FG            3.5 -116.009281   
17           Donovan Mitchell        Assists            6.5 -102.040816   
18           Donovan Mitchell         Blocks            0.5  236.000000   
19           Donovan Mitchell  Double+Double            0.5  335.000000   
20           Donovan Mitchell         Points           28.5 -108.108108   
21           Donovan Mitchell  Pts+Rebs+Asts           40.5 -101.010101   
22           Donovan Mitchell       Rebounds            4.5 -123.001230   
23           Donovan Mitchell  Steals+Blocks            1.5 -121.065375   
24           Donovan Mitchell      Turnovers            3.5  144.000000   
25                Evan Mobley     3 Point FG            0.5  237.000000   
26                Evan Mobley        Assists            2.5  123.000000   
27                Evan Mobley         Blocks            1.5  108.000000   
28                Evan Mobley  Double+Double            0.5  256.000000   
29                Evan Mobley         Points           14.5 -148.148148   
30                Evan Mobley  Pts+Rebs+Asts           24.5 -126.103405   
31                Evan Mobley       Rebounds            7.5 -113.122172   
32                Evan Mobley  Steals+Blocks            2.5  103.000000   
33                Evan Mobley      Turnovers            1.5 -173.010381   
34               Franz Wagner     3 Point FG            1.5 -102.040816   
35               Franz Wagner        Assists            4.5  123.000000   
36               Franz Wagner         Blocks            0.5  184.000000   
37               Franz Wagner         Points           16.5 -113.122172   
38               Franz Wagner  Pts+Rebs+Asts           25.5 -110.011001   
39               Franz Wagner       Rebounds            4.5 -124.069479   
40               Franz Wagner  Steals+Blocks            1.5  107.000000   
41               Franz Wagner      Turnovers            1.5 -173.010381   
42              Jarrett Allen        Assists            1.5 -120.048019   
43              Jarrett Allen         Blocks            1.5  142.000000   
44              Jarrett Allen  Double+Double            0.5 -128.040973   
45              Jarrett Allen         Points           14.5 -103.092784   
46              Jarrett Allen  Pts+Rebs+Asts           26.5 -105.042017   
47              Jarrett Allen       Rebounds           10.5 -104.058273   
48              Jarrett Allen  Steals+Blocks            1.5 -171.232877   
49              Jarrett Allen      Turnovers            1.5  104.000000   
50             Paolo Banchero     3 Point FG            1.5  178.000000   
51             Paolo Banchero        Assists            2.5 -162.074554   
52             Paolo Banchero         Block

# Scraping NHL

In [140]:
nhl_link = 'https://www.pinnacle.com/en/hockey/nhl/matchups#period:0'

In [141]:
driver.get(nhl_link)
time.sleep(5)

In [142]:
content = driver.page_source
soup = BeautifulSoup(content, "html.parser")

In [143]:
results = soup.find_all("div", class_="style_metadata__1FIzs")

In [144]:
nhl_final_df = manipulation(results)
nhl_final_df = converting_odds(nhl_final_df)

In [145]:
nhl_final_df['league'] = 'nhl'
nhl_final_df

player_name      prop_type  pinnacle_line  over_price  \
0                 Adam Fox        Assists            0.5 -113.122172   
1                 Adam Fox         Points            0.5 -147.058824   
2                 Adam Fox  Shots On Goal            1.5 -175.131349   
3        Alexis Lafreniere          Goals            0.5  313.000000   
4               Anders Lee        Assists            0.5  199.000000   
5               Anders Lee          Goals            0.5  193.000000   
6               Anders Lee         Points            0.5 -128.040973   
7               Anders Lee  Shots On Goal            2.5  132.000000   
8           Artemi Panarin        Assists            0.5 -146.198830   
9           Artemi Panarin          Goals            0.5  220.000000   
10          Artemi Panarin         Points            0.5 -272.479564   
11          Artemi Panarin  Shots On Goal            2.5 -115.074799   
12            Brock Nelson        Assists            0.5  144.000000   
13            Brock Nelson          Goals            0.5  189.000000   
14            Brock Nelson         Points            0.5 -164.203612   
15            Brock Nelson  Shots On Goal            2.5  133.000000   
16           Chris Kreider        Assists            0.5  168.000000   
17           Chris Kreider          Goals            0.5  166.000000   
18           Chris Kreider         Points            0.5 -156.006240   
19           Chris Kreider  Shots On Goal            2.5 -121.065375   
20            Jacob Trouba  Shots On Goal            2.5  119.000000   
21     Jean-Gabriel Pageau        Assists            0.5  195.000000   
22     Jean-Gabriel Pageau          Goals            0.5  297.000000   
23     Jean-Gabriel Pageau         Points            0.5  104.000000   
24             Josh Bailey        Assists            0.5  210.000000   
25             Josh Bailey          Goals            0.5  395.000000   
26             Josh Bailey         Points            0.5  129.000000   
27             Kaapo Kakko          Goals            0.5  300.000000   
28           Kyle Palmieri        Assists            0.5  213.000000   
29           Kyle Palmieri          Goals            0.5  279.000000   
30           Kyle Palmieri         Points            0.5  105.000000   
31           Kyle Palmieri  Shots On Goal            2.5  130.000000   
32           Mathew Barzal        Assists            0.5 -148.148148   
33           Mathew Barzal          Goals            0.5  259.000000   
34           Mathew Barzal         Points            0.5 -229.357798   
35          Mika Zibanejad        Assists            0.5  104.000000   
36          Mika Zibanejad          Goals            0.5  183.000000   
37          Mika Zibanejad         Points            0.5 -214.132762   
38          Mika Zibanejad  Shots On Goal            3.5  127.000000   
39             Noah Dobson        Assists            0.5  117.000000   
40             Noah Dobson          Goals            0.5  377.000000   
41             Noah Dobson         Points            0.5 -129.032258   
42        Oliver Wahlstrom          Goals            0.5  324.000000   
43        Oliver Wahlstrom  Shots On Goal            2.5  107.000000   
44        Vincent Trocheck        Assists            0.5  145.000000   
45        Vincent Trocheck          Goals            0.5  262.000000   
46        Vincent Trocheck         Points            0.5 -131.061599   
47        Vincent Trocheck  Shots On Goal            2.5 -149.031297   
48             Zach Parise          Goals            0.5  328.000000   
49          Brayden Schenn        Assists            0.5  103.000000   
50          Brayden Schenn          Goals            0.5  240.000000   
51          Brayden Schenn         Points            0.5 -186.219739   
52          Connor McDavid        Assists            1.5  159.000000   
53          Connor McDavid          Goals            0.5  110.000000   
54          Connor McDavid         Points            1.

# Scraping MLB

In [146]:
mlb_link = 'https://www.pinnacle.com/en/baseball/mlb/matchups#period:0'

In [147]:
driver.get(mlb_link)
time.sleep(5)

In [148]:
content = driver.page_source
soup = BeautifulSoup(content, "html.parser")

In [149]:
results = soup.find_all("div", class_="style_metadata__1FIzs")

In [150]:
mlb_final_df = manipulation(results)
mlb_final_df = converting_odds(mlb_final_df)

In [151]:
mlb_final_df['league'] = 'mlb'
mlb_final_df

player_name    prop_type  pinnacle_line  over_price  under_price  \
0          Alec Bohm  Total Bases            1.5  155.000000  -216.450216   
1       Alex Bregman  Total Bases            0.5 -196.078431   142.000000   
2      Brandon Marsh  Total Bases            0.5 -111.111111  -119.047619   
3       Bryce Harper    Home Runs            0.5  329.000000  -617.283951   
4       Bryce Harper  Total Bases            1.5  107.000000  -142.045455   
5       Bryson Stott  Total Bases            0.5 -159.235669   118.000000   
6     Chas McCormick  Total Bases            0.5 -108.108108  -122.100122   
7      J.T. Realmuto  Total Bases            1.5  133.000000  -181.159420   
8        Jean Segura  Total Bases            0.5 -183.150183   134.000000   
9        Jeremy Pena  Total Bases            0.5 -198.019802   143.000000   
10       Jose Altuve  Total Bases            0.5 -193.050193   140.000000   
11    Kyle Schwarber    Home Runs            0.5  279.000000  -478.468900   
12    Kyle Schwarber  Total Bases            0.5 -151.057402   113.000000   
13       Kyle Tucker  Total Bases            0.5 -170.068027   126.000000   
14  Martin Maldonado  Total Bases            0.5  156.000000  -219.298246   
15  Nick Castellanos  Total Bases            0.5 -197.238659   143.000000   
16      Rhys Hoskins    Home Runs            0.5  392.000000  -826.446281   
17      Rhys Hoskins  Total Bases            0.5 -184.162063   135.000000   
18      Trey Mancini  Total Bases            0.5 -138.121547   104.000000   
19    Yordan Alvarez    Home Runs            0.5  321.000000  -595.238095   
20    Yordan Alvarez  Total Bases            1.5  136.000000  -186.219739   
21      Yuli Gurriel  Total Bases            0.5 -201.207243   145.000000   

   league  
0     mlb  
1     mlb  
2     mlb  
3     mlb  
4     mlb  
5     mlb  
6     mlb  
7     mlb  
8     mlb  
9     mlb  
10    mlb  
11    mlb  
12    mlb  
13    mlb  
14    mlb  
15    mlb  
16    mlb  
17    mlb  
18    mlb  
19    mlb  
20    mlb  
21    mlb

In [152]:
final_df = nfl_final_df.append(mlb_final_df).append(nhl_final_df).append(nba_final_df)
final_df['player_name'] = final_df['player_name'].str.lower()
final_df['prop_type'] = final_df['prop_type'].str.lower()
final_df

player_name        prop_type  pinnacle_line  over_price  \
0                  cade otton    td receptions            0.5  360.000000   
1                chris godwin   long reception           22.5 -113.122172   
2                chris godwin    receiving yds           71.5 -114.025086   
3                chris godwin    td receptions            0.5  147.000000   
4              devin duvernay    td receptions            0.5  395.000000   
5                 gus edwards          rush td            0.5  177.000000   
6                 gus edwards         rush yds           45.5 -120.048019   
7               justin tucker   kicking points            7.5 -114.025086   
8               lamar jackson      completions           19.5 -106.044539   
9               lamar jackson    interceptions            0.5 -127.064803   
10              lamar jackson  long completion           33.5 -114.025086   
11              lamar jackson    pass attempts           29.5 -101.010101   
12              lamar jackson         pass yds          213.5 -114.025086   
13              lamar jackson          rush td            0.5  208.000000   
14              lamar jackson         rush yds           61.5 -116.009281   
15              lamar jackson        td passes            1.5  143.000000   
16          leonard fournette   long reception           13.5 -102.040816   
17          leonard fournette    receiving yds           28.5 -113.122172   
18          leonard fournette          rush td            0.5  154.000000   
19          leonard fournette         rush yds           49.5  104.000000   
20          leonard fournette    td receptions            0.5  335.000000   
21                 mike evans   long reception           24.5 -122.100122   
22                 mike evans    receiving yds           68.5 -113.122172   
23                 mike evans    td receptions            0.5  132.000000   
24              rachaad white   long reception            8.5 -121.065375   
25              rachaad white    receiving yds           12.5  104.000000   
26                  tom brady    interceptions            0.5  106.000000   
27                  tom brady  long completion           37.5 -119.047619   
28                  tom brady         pass yds          279.5 -116.009281   
29                  tom brady        td passes            1.5 -124.069479   
0                   alec bohm      total bases            1.5  155.000000   
1                alex bregman      total bases            0.5 -196.078431   
2               brandon marsh      total bases            0.5 -111.111111   
3                bryce harper        home runs            0.5  329.000000   
4                bryce harper      total bases            1.5  107.000000   
5                bryson stott      total bases            0.5 -159.235669   
6              chas mccormick      total bases            0.5 -108.108108   
7               j.t. realmuto      total bases            1.5  133.000000   
8                 jean segura      total bases            0.5 -183.150183   
9                 jeremy pena      total bases            0.5 -198.019802   
10                jose altuve      total bases            0.5 -193.050193   
11             kyle schwarber        home runs            0.5  279.000000   
12             kyle schwarber      total bases            0.5 -151.057402   
13                kyle tucker      total bases            0.5 -170.068027   
14           martin maldonado      total bases            0.5  156.000000   
15           nick castellanos      total bases            0.5 -197.238659   
16               rhys hoskins        home runs            0.5  392.000000   
17               rhys hoskins      total bases            0.5 -184.162063   
18               trey mancini      total bases            0.5 -138.121547   
19             yordan alvarez        home runs            0.5  321.000000   
20             yordan alvarez      total bases            1.5  136.000000   
21            

# PrizePicks

In [153]:
PrizePicksLink = 'https://api.prizepicks.com/projections'
driver.get(PrizePicksLink)
time.sleep(5)

In [154]:
content = driver.page_source
soup = BeautifulSoup(content, "html.parser")
data = json.loads(soup.text)
pp = pd.json_normalize(data['included'])
pp1 = pp[pp['type'] == 'new_player']
pp2 = pd.json_normalize(data['data'])

pp1 = pp1.rename(columns = {'id' : 'playerid'})
pp2 = pp2.rename(columns = {'relationships.new_player.data.id' : 'playerid'})

mergedDF = pp2.merge(pp1, on = 'playerid', how = 'left')
fdf = mergedDF[['attributes.line_score','attributes.stat_type','attributes.name','attributes.league']]
fdf.rename(columns={'attributes.line_score':'pp_line', 'attributes.stat_type':'prop_type', 'attributes.name':'player_name', 
                   'attributes.league':'league'}, inplace=True)
fdf['player_name'] = fdf['player_name'].str.lower()
fdf['league'] = fdf['league'].str.lower()
fdf['prop_type'] = fdf['prop_type'].str.lower()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [155]:
# changing prop types to match pinnacle
# keep adding to this until complete

# MLB PROPS
fdf['prop_type'] = np.where(fdf['prop_type']=='earned runs allowed', 'earned runs',
                           np.where(fdf['prop_type']=='strikeouts', 'total strikeouts',
                                   fdf['prop_type']))

#NFL Props
fdf['prop_type'] = np.where(fdf['prop_type']=='pass yards', 'pass yds', 
                           np.where(fdf['prop_type']=='receiving yards', 'receiving yds', 
                                    np.where(fdf['prop_type']=='rush yards', 'rush yds',
                                            fdf['prop_type'])))

#NHL Props
fdf['prop_type'] = np.where(fdf['prop_type']=='goalie saves', 'saves', fdf['prop_type'])


#NBA Props
fdf['prop_type'] = np.where(fdf['prop_type']=='3-pt made', '3 point fg', 
                           np.where(fdf['prop_type']=='blocked shots', 'blocks', 
                                    np.where(fdf['prop_type']=='blks+stls', 'steals+blocks',
                                            fdf['prop_type'])))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

# Merging

In [156]:
merged_df = fdf.merge(final_df, how = 'left', on = ['player_name', 'prop_type','league'])

In [157]:
merged_df.dropna(inplace = True)
merged_df['min_price'] = np.where(merged_df['over_price']<=merged_df['under_price'], 
                                merged_df['over_price'], merged_df['under_price'])

merged_df['favored'] = np.where(merged_df['min_price']==merged_df['over_price'], 'over', 'under')
    
merged_df['edge'] = np.where((merged_df['pinnacle_line']<merged_df['pp_line'])&(merged_df['favored']=='under'), 'edge',
                        np.where((merged_df['pinnacle_line']>merged_df['pp_line'])&(merged_df['favored']=='over'), 'edge',
                        np.where(merged_df['pinnacle_line']==merged_df['pp_line'], 'equal',
                        np.where((merged_df['pinnacle_line']<merged_df['pp_line'])&(merged_df['favored']=='over'), 'buffer',
                        np.where((merged_df['pinnacle_line']>merged_df['pp_line'])&(merged_df['favored']=='under'), 'buffer', 'else')))))
pd.set_option('display.max_rows',None)
merged_df.sort_values(by = 'min_price', ascending = True)

pp_line       prop_type          player_name league  pinnacle_line  \
984      2.0   shots on goal  ryan nugent-hopkins    nhl            2.5   
965      2.0   shots on goal         brock nelson    nhl            2.5   
964      2.0   shots on goal           anders lee    nhl            2.5   
413      4.0         assists           jaden ivey    nba            4.5   
390      4.0        rebounds     bojan bogdanovic    nba            3.5   
974      2.0   shots on goal        kyle palmieri    nhl            2.5   
972      2.0   shots on goal             adam fox    nhl            1.5   
981      2.0   shots on goal        ryan o'reilly    nhl            2.5   
968      3.5   shots on goal       mika zibanejad    nhl            3.5   
986      3.0   shots on goal       leon draisaitl    nhl            3.5   
980      4.0   shots on goal         evander kane    nhl            3.5   
985      3.0   shots on goal   vladimir tarasenko    nhl            2.5   
982      3.0   shots on goal           zach hyman    nhl            2.5   
971      2.0   shots on goal         jacob trouba    nhl            2.5   
979      3.0   shots on goal         jordan kyrou    nhl            2.5   
369      3.0      3 point fg           trae young    nba            3.5   
562     23.5          points        pascal siakam    nba           23.5   
660     14.5          points         kelly olynyk    nba           14.5   
967      3.0   shots on goal     vincent trocheck    nhl            2.5   
553     27.5          points          joel embiid    nba           27.5   
82      14.5          points          evan mobley    nba           14.5   
949      0.5         assists        mathew barzal    nhl            0.5   
19      16.0          points         cole anthony    nba           16.5   
975      2.0   shots on goal     oliver wahlstrom    nhl            2.5   
378      5.0        rebounds      cade cunningham    nba            4.5   
564     14.5          points        tobias harris    nba           14.5   
377      7.0         assists      cade cunningham    nba            6.5   
396      8.0        rebounds         john collins    nba            7.5   
976      2.5   shots on goal        evan bouchard    nhl            2.5   
951      0.5         assists       mika zibanejad    nhl            0.5   
333     12.5   receiving yds        rachaad white    nfl           12.5   
239      0.5     total bases         trey mancini    mlb            0.5   
555     17.5          points        fred vanvleet    nba           17.5   
386      7.5         assists      dejounte murray    nba            7.5   
314     51.5        rush yds    leonard fournette    nfl           49.5   
959      0.5          points        adam henrique    nhl            0.5   
385      6.5        rebounds      dejounte murray    nba            6.5   
409     10.0        rebounds         clint capela    nba           10.5   
958      0.5          points         alex killorn    nhl            0.5   
650     23.5          points          jalen green    nba           23.5   
405      0.5         assists         clint capela    nba            0.5   
399     16.5          points         john collins    nba           16.5   
417      1.5      3 point fg      de'andre hunter    nba            1.5   
375      1.5      3 point fg      cade cunningham    nba            1.5   
655     18.5          points      jordan clarkson    nba           18.5   
411     14.5          points           jaden ivey    nba           14.5   
138     13.5          points        jarrett allen    nba           14.5   
558     23.5          points         james harden    nba           22.5   
950      0.5          points           anders lee    nhl            0.5   
380     19.5          points      cade cunningham    nba           19.5   
404      9.5          points       isaiah stewart    nba            9.5   
978      3.5   shots on goal       connor mcdavid    nhl            3.5   
953      0.5         assists  

# Dropping Buffered Lines

In [158]:
equal_lines_df = merged_df.loc[merged_df['edge']!='buffer']
equal_lines_df.sort_values(by = 'min_price', ascending = True)

pp_line       prop_type          player_name league  pinnacle_line  \
968      3.5   shots on goal       mika zibanejad    nhl            3.5   
562     23.5          points        pascal siakam    nba           23.5   
660     14.5          points         kelly olynyk    nba           14.5   
553     27.5          points          joel embiid    nba           27.5   
82      14.5          points          evan mobley    nba           14.5   
949      0.5         assists        mathew barzal    nhl            0.5   
564     14.5          points        tobias harris    nba           14.5   
976      2.5   shots on goal        evan bouchard    nhl            2.5   
951      0.5         assists       mika zibanejad    nhl            0.5   
333     12.5   receiving yds        rachaad white    nfl           12.5   
555     17.5          points        fred vanvleet    nba           17.5   
239      0.5     total bases         trey mancini    mlb            0.5   
314     51.5        rush yds    leonard fournette    nfl           49.5   
386      7.5         assists      dejounte murray    nba            7.5   
959      0.5          points        adam henrique    nhl            0.5   
385      6.5        rebounds      dejounte murray    nba            6.5   
958      0.5          points         alex killorn    nhl            0.5   
650     23.5          points          jalen green    nba           23.5   
399     16.5          points         john collins    nba           16.5   
405      0.5         assists         clint capela    nba            0.5   
375      1.5      3 point fg      cade cunningham    nba            1.5   
417      1.5      3 point fg      de'andre hunter    nba            1.5   
655     18.5          points      jordan clarkson    nba           18.5   
558     23.5          points         james harden    nba           22.5   
411     14.5          points           jaden ivey    nba           14.5   
380     19.5          points      cade cunningham    nba           19.5   
950      0.5          points           anders lee    nhl            0.5   
404      9.5          points       isaiah stewart    nba            9.5   
953      0.5         assists        robert thomas    nhl            0.5   
978      3.5   shots on goal       connor mcdavid    nhl            3.5   
373     25.5          points           trae young    nba           25.5   
653     20.5          points     kevin porter jr.    nba           20.5   
572     17.5          points         tyrese maxey    nba           17.5   
415      3.5        rebounds      de'andre hunter    nba            3.5   
150     14.5          points   wendell carter jr.    nba           14.5   
372      9.5         assists           trae young    nba            9.5   
43      28.5          points     donovan mitchell    nba           28.5   
219      0.5     total bases       chas mccormick    mlb            0.5   
652     19.5          points      lauri markkanen    nba           19.5   
969      2.5   shots on goal        chris kreider    nhl            2.5   
392      2.5      3 point fg     bojan bogdanovic    nba            2.5   
956      1.5          points       leon draisaitl    nhl            1.5   
955      0.5          points         justin faulk    nhl            0.5   
408      9.5          points         clint capela    nba            9.5   
388      1.5      3 point fg      dejounte murray    nba            1.5   
954      0.5         assists  ryan nugent-hopkins    nhl            0.5   
197     12.5          points        terrence ross    nba           12.5   
236      0.5     total bases        brandon marsh    mlb            0.5   
568     17.5          points       gary trent jr.    nba           17.5   
957      0.5         assists       steven stamkos    nhl            0.5   
393     18.5          points     bojan bogdanovic    nba           18.5   
952      0.5         assists             adam fox    nhl            0.5   
416     15.5          points  

# Only Lines With Edges

In [159]:
edge_lines_df = merged_df.loc[merged_df['edge']=='edge']
edge_lines_df.sort_values(by = 'min_price', ascending = True)

pp_line      prop_type        player_name league  pinnacle_line  \
314     51.5       rush yds  leonard fournette    nfl           49.5   
558     23.5         points       james harden    nba           22.5   
316     29.5  receiving yds  leonard fournette    nfl           28.5   
286     60.5       rush yds      lamar jackson    nfl           61.5   
662     11.5         points        mike conley    nba           12.5   
177     14.5         points       caris levert    nba           15.5   

     over_price  under_price   min_price favored  edge  
314  104.000000  -137.174211 -137.174211   under  edge  
558 -103.092784  -129.032258 -129.032258   under  edge  
316 -113.122172  -117.096019 -117.096019   under  edge  
286 -116.009281  -114.025086 -116.009281    over  edge  
662 -116.009281  -113.122172 -116.009281    over  edge  
177 -115.074799  -115.074799 -115.074799    over  edge